# Introduction
* Maxcruz의 Complaint 데이터와 Review 데이터를 분석하여 **Maxcruz와 비중있는 경쟁차량을 선정**한다.
    1. Maxcruz를 구입한 고객들 리뷰에서 다른 경쟁차량이 얼마나 언급되는지
    2. 경쟁차량을 구입한 고객 리뷰에서 Maxcruz가 얼마나 언급되는지

In [1]:
import urllib.request, json 
import pandas as pd
import numpy as np
import os
import nltk, gensim
from nltk import word_tokenize

C:\Users\kb910\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
os.getcwd()

'C:\\Users\\kb910\\Google 드라이브\\경배끄10\\인턴\\python_code\\리뷰'

# 리뷰데이터

In [38]:
os.chdir('C:\\Users\\kb910\\Google 드라이브\\경배끄10\\인턴\\csv파일\\리뷰들\\SUV리뷰')
data = open('all_suv_review(15~17).json').read()
suv_review1= pd.DataFrame(json.loads(data))
suv_review1['model_name'].unique()

array(['murano', 'cx-9', '4runner', 'grand-cherokee', 'qx50', 'pathfinder',
       'flex', 'traverse', 'acadia', 'highlander', 'explorer', 'pilot',
       'durango'], dtype=object)

In [39]:
os.chdir('C:\\Users\\kb910\\Google 드라이브\\경배끄10\\인턴\\csv파일\\리뷰들\\SUV리뷰')
data = open('all_suv_review2(15~17).json').read()
suv_review2= pd.DataFrame(json.loads(data))
suv_review2['model_name'].unique()

array(['edge', 'santa-fe', 'sorento'], dtype=object)

In [40]:
last_suv_review = pd.concat([suv_review1,suv_review2])

# 컴플레인 데이터

In [41]:
os.chdir('C:\\Users\\kb910\\Google 드라이브\\경배끄10\\인턴\\csv파일\\complain')
data = open('real_all_suv_compalin(15~17).json').read()
suv_complain = pd.DataFrame(json.loads(data))
suv_complain['model_name'].unique()

array(['durango', 'explorer', 'edge', 'traverse', 'acadia', 'flex', 'cx-9',
       'murano', 'pathfinder', '4runner', 'highlander', 'pilot',
       'santa_fe', 'qx50', 'sorento'], dtype=object)

In [42]:
os.chdir('C:\\Users\\kb910\\Google 드라이브\\경배끄10\\인턴\\csv파일\\complain')
data = pd.read_csv('Santafe_review_16.csv')
data2 = pd.read_csv('Santafe_review_17.csv')
Complain_data = pd.concat([data, data2])
Complain_data = Complain_data.drop(['Unnamed: 0'],axis=1)
Complain_data.head(5)

,B_category,S_category,city,country,date,sub_title,text,title,year
0,fuel_system,fuel_propulsion_system,NC,USA,28-Jun,"8,000 miles",I bought a 2016 Santa Fe sport at carmax 8000 ...,Santa Fe,2016
1,seat_belts_air_bags,seat_belts-front-retractor,Sound,USA,03-Sep,miles,2016 Hyundai sante fe involved in a near head ...,Santa Fe,2016
2,wheels_hubs,wheels,TX,USA,25-Mar,"3,400 miles",The SUV 2016 come equipped with Continental 23...,Santa Fe,2016
3,body_paint,structure-body,Haven,USA,04-Dec,"13,640 miles",The entire panarama sunroof exploded on my 201...,Santa Fe,2016
4,body_paint,structure,VA,USA,06-Oct,900 miles,The contact owns a 2016 Hyundai Santa Fe. The ...,Santa Fe,2015


In [45]:
Complain_data['model_name']='santa-fe'

## 데이터 합치기

In [46]:
last_suv_review_text = last_suv_review[['model_name','text']]
suv_complain_text = suv_complain[['model_name','text']]
Complain_data = Complain_data[['model_name','text']]

In [47]:
merge_suv = pd.concat([last_suv_review_text, suv_complain_text, Complain_data])

In [48]:
len(merge_suv[merge_suv['model_name']=='santa-fe'])

146

## 경쟁차량리뷰에서  Maxcruz 비중

In [49]:
def SUV_to_Maxcruz(table):
    
    model_list = merge_suv['model_name'].unique()
    
    for model in model_list:
        table = merge_suv[merge_suv['model_name']==model]
        voca = ['santa-fe','santa','Santa','Santa Fe','Santa-Fe']
        test = [ str(x) for x in table['text'].values.tolist() ] 

        from sklearn.feature_extraction.text import CountVectorizer
        vectorizer = CountVectorizer(vocabulary=voca)
        context = test
        X = vectorizer.fit_transform(context)
        TDM_matrix = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

        for column in TDM_matrix.columns:
            TDM_matrix.loc[TDM_matrix[column] != 0, column] = 1

        TDM_matrix.loc[('sum_count')] = [ TDM_matrix[x].sum() for x in TDM_matrix.columns ]

        score = round(TDM_matrix.loc[('sum_count')].sum() / len(table)*100,3)
        
        print(str(model)+"의 Maxcruz 경쟁 비중 : "+str(score)+"%"+'  / 리뷰 counting :  '+str(TDM_matrix.loc[('sum_count')].sum()) +  '   / 전체리뷰개수 :'+str(len(table)))
        print('\n')

In [50]:
SUV_to_Maxcruz(merge_suv)

murano의 Maxcruz 경쟁 비중 : 0.344%  / 리뷰 counting :  1   / 전체리뷰개수 :291


cx-9의 Maxcruz 경쟁 비중 : 0.893%  / 리뷰 counting :  1   / 전체리뷰개수 :112


4runner의 Maxcruz 경쟁 비중 : 0.0%  / 리뷰 counting :  0   / 전체리뷰개수 :167


grand-cherokee의 Maxcruz 경쟁 비중 : 0.73%  / 리뷰 counting :  1   / 전체리뷰개수 :137


qx50의 Maxcruz 경쟁 비중 : 0.0%  / 리뷰 counting :  0   / 전체리뷰개수 :26


pathfinder의 Maxcruz 경쟁 비중 : 1.17%  / 리뷰 counting :  2   / 전체리뷰개수 :171


flex의 Maxcruz 경쟁 비중 : 0.0%  / 리뷰 counting :  0   / 전체리뷰개수 :49


traverse의 Maxcruz 경쟁 비중 : 0.645%  / 리뷰 counting :  1   / 전체리뷰개수 :155


acadia의 Maxcruz 경쟁 비중 : 3.65%  / 리뷰 counting :  5   / 전체리뷰개수 :137


highlander의 Maxcruz 경쟁 비중 : 0.272%  / 리뷰 counting :  1   / 전체리뷰개수 :368


explorer의 Maxcruz 경쟁 비중 : 0.398%  / 리뷰 counting :  3   / 전체리뷰개수 :754


pilot의 Maxcruz 경쟁 비중 : 0.542%  / 리뷰 counting :  2   / 전체리뷰개수 :369


durango의 Maxcruz 경쟁 비중 : 0.0%  / 리뷰 counting :  0   / 전체리뷰개수 :133


edge의 Maxcruz 경쟁 비중 : 1.185%  / 리뷰 counting :  5   / 전체리뷰개수 :422


santa-fe의 Maxcruz 경쟁 비중 : 45.89%  

In [51]:
# acadia / sorento / pathfinder / cx-9 / edge / traverse / murano / pilot / highlander / 

## Maxcruz리뷰에서 경쟁차량 파악

In [56]:
def Maxcruz_to_SUV(table):
    
    voca_list = ['highlander','durango','cx','pilot','pathfinder','4runner','flex','traverse','acadia','explorer','murano','cherokee','sorento','edge']
    
    for voca in voca_list:

        test = [ str(x) for x in table['text'].values.tolist() ] 

        from sklearn.feature_extraction.text import CountVectorizer
        vectorizer = CountVectorizer(vocabulary=[voca])
        context = test
        X = vectorizer.fit_transform(context)
        TDM_matrix = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

        for column in TDM_matrix.columns:
            TDM_matrix.loc[TDM_matrix[column] != 0, column] = 1

        TDM_matrix.loc[('sum_count')] = [ TDM_matrix[x].sum() for x in TDM_matrix.columns ]

        score = round(TDM_matrix.loc[('sum_count')].sum() / len(table)*100,3)

        print("Maxcruz의" +str(voca)+ "경쟁 비중 : "+str(score)+"%"  '/ 리뷰 counting :  '+str(TDM_matrix.loc[('sum_count')].sum()) +  '   / 전체리뷰개수 :'+str(len(table)))
        print('\n')


In [57]:
Maxcruz_to_SUV(merge_suv[merge_suv['model_name']=='santa-fe'])

Maxcruz의highlander경쟁 비중 : 3.425%/ 리뷰 counting :  5   / 전체리뷰개수 :146


Maxcruz의durango경쟁 비중 : 0.0%/ 리뷰 counting :  0   / 전체리뷰개수 :146


Maxcruz의cx경쟁 비중 : 0.0%/ 리뷰 counting :  0   / 전체리뷰개수 :146


Maxcruz의pilot경쟁 비중 : 5.479%/ 리뷰 counting :  8   / 전체리뷰개수 :146


Maxcruz의pathfinder경쟁 비중 : 0.685%/ 리뷰 counting :  1   / 전체리뷰개수 :146


Maxcruz의4runner경쟁 비중 : 0.0%/ 리뷰 counting :  0   / 전체리뷰개수 :146


Maxcruz의flex경쟁 비중 : 0.0%/ 리뷰 counting :  0   / 전체리뷰개수 :146


Maxcruz의traverse경쟁 비중 : 1.37%/ 리뷰 counting :  2   / 전체리뷰개수 :146


Maxcruz의acadia경쟁 비중 : 0.685%/ 리뷰 counting :  1   / 전체리뷰개수 :146


Maxcruz의explorer경쟁 비중 : 1.37%/ 리뷰 counting :  2   / 전체리뷰개수 :146


Maxcruz의murano경쟁 비중 : 0.685%/ 리뷰 counting :  1   / 전체리뷰개수 :146


Maxcruz의cherokee경쟁 비중 : 0.0%/ 리뷰 counting :  0   / 전체리뷰개수 :146


Maxcruz의sorento경쟁 비중 : 2.055%/ 리뷰 counting :  3   / 전체리뷰개수 :146


Maxcruz의edge경쟁 비중 : 1.37%/ 리뷰 counting :  2   / 전체리뷰개수 :146


